In [2]:
import pandas as pd

# Ruta completa y correcta
archivo = r"C:\Users\guerr\Documents\AnalisisDeDatos\Dahell\data\ordenes_productos-historial.xlsx"

# Intentar cargar
try:
    df = pd.read_excel(archivo, engine='openpyxl')
    print("✅ Archivo cargado con éxito")
except FileNotFoundError:
    print("❌ No se encontró el archivo, verifica la ruta.")
except Exception as e:
    print("❌ Otro error:", e)

df.head()  # Mostrar las primeras filas del DataFrame
df.info()

✅ Archivo cargado con éxito
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 53 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   FECHA DE REPORTE                480 non-null    object 
 1   ID                              480 non-null    int64  
 2   HORA                            480 non-null    object 
 3   FECHA                           480 non-null    object 
 4   NOMBRE CLIENTE                  480 non-null    object 
 5   TELÉFONO                        480 non-null    int64  
 6   EMAIL                           366 non-null    object 
 7   TIPO DE IDENTIFICACION          0 non-null      float64
 8   NRO DE IDENTIFICACION           0 non-null      float64
 9   NÚMERO GUIA                     325 non-null    object 
 10  ESTATUS                         480 non-null    object 
 11  TIPO DE ENVIO                   480 non-null    object 
 12  DEPARTAM

In [3]:
# Eliminar columnas irrelevantes
columnas_para_eliminar = [
    'TELÉFONO', 'EMAIL', 'TIPO DE IDENTIFICACION', 'NRO DE IDENTIFICACION',
    'DIRECCION', 'FECHA DE REPORTE', 'ID DE ORDEN DE TIENDA', 'TAGS',
    'HORA DE NOVEDAD', 'HORA DE SOLUCIÓN', 'HORA DE ÚLTIMO MOVIMIENTO',
    'OBSERVACIÓN', 'PRODUCTO ID', 'SKU', 'VARIACION ID', 'TIPO DE TIENDA', 'TIENDA',
    'NOTAS', 'COMISION', 'COSTO DEVOLUCION FLETE', 'VARIACION',
    'NOVEDAD', 'FUE SOLUCIONADA LA NOVEDAD', 'FECHA DE NOVEDAD',
    'SOLUCIÓN', 'FECHA DE SOLUCIÓN', 'FECHA DE ÚLTIMO MOVIMIENTO',
    'ÚLTIMO MOVIMIENTO', 'CONCEPTO ÚLTIMO MOVIMIENTO',
    'UBICACIÓN DE ÚLTIMO MOVIMIENTO', 'VENDEDOR',
    'CONTADOR DE INDEMNIZACIONES', 'CONCEPTO ÚLTIMA INDENMIZACIÓN',
    'TIPO DE TIENDA', 'TIENDA', 'NUMERO DE PEDIDO DE TIENDA',
    'FECHA GUIA GENERADA', 'NÚMERO GUIA'
]
df.drop(columns=columnas_para_eliminar, inplace=True, errors='ignore')

# Verificar tipos de datos
print("Tipos de datos:")
print(df.dtypes)


Tipos de datos:
ID                                int64
HORA                             object
FECHA                            object
NOMBRE CLIENTE                   object
ESTATUS                          object
TIPO DE ENVIO                    object
DEPARTAMENTO DESTINO             object
CIUDAD DESTINO                   object
TRANSPORTADORA                   object
TOTAL DE LA ORDEN                 int64
GANANCIA                        float64
PRECIO FLETE                    float64
% COMISION DE LA PLATAFORMMA      int64
PRECIO PROVEEDOR                  int64
PRECIO PROVEEDOR X CANTIDAD       int64
PRODUCTO                         object
CANTIDAD                          int64
dtype: object


In [4]:
import pandas as pd

# Asegúrate de tener el DataFrame cargado como 'df'

# 1. Convertir columna FECHA a datetime
df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')

# 2. Convertir HORA a tipo datetime.time
# Si 'HORA' viene en formato como "14:25", este código funcionará:
df['HORA'] = pd.to_datetime(df['HORA'], format='%H:%M', errors='coerce').dt.time

# 3. Asegurarse de que valores numéricos estén en float (para evitar divisiones erróneas)
columnas_a_float = [
    'TOTAL DE LA ORDEN', 'GANANCIA', 'PRECIO FLETE',
    '% COMISION DE LA PLATAFORMMA', 'PRECIO PROVEEDOR',
    'PRECIO PROVEEDOR X CANTIDAD', 'CANTIDAD'
]

for col in columnas_a_float:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [5]:
# Verificar nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())


Valores nulos por columna:
ID                                0
HORA                              0
FECHA                             0
NOMBRE CLIENTE                    0
ESTATUS                           0
TIPO DE ENVIO                     0
DEPARTAMENTO DESTINO              0
CIUDAD DESTINO                    0
TRANSPORTADORA                    0
TOTAL DE LA ORDEN                 0
GANANCIA                        264
PRECIO FLETE                      0
% COMISION DE LA PLATAFORMMA      0
PRECIO PROVEEDOR                  0
PRECIO PROVEEDOR X CANTIDAD       0
PRODUCTO                          0
CANTIDAD                          0
dtype: int64


In [6]:
# Verificar duplicados
duplicados = df.duplicated().sum()
print(f"\nCantidad de filas duplicadas: {duplicados}")


Cantidad de filas duplicadas: 1


In [7]:
# Eliminar duplicados exactos
df = df.drop_duplicates()


In [8]:
# Día de la semana (0 = lunes, 6 = domingo)
df['DIA_SEMANA'] = df['FECHA'].dt.day_name()

# Mes
df['MES'] = df['FECHA'].dt.month_name()

# Año
df['AÑO'] = df['FECHA'].dt.year

# Franja horaria (mañana: 6-11, tarde: 12-17, noche: 18-23, madrugada: 0-5)
def clasificar_franja_horaria(hora):
    if pd.isnull(hora):
        return 'Desconocido'
    h = hora.hour
    if 6 <= h < 12:
        return 'Mañana'
    elif 12 <= h < 18:
        return 'Tarde'
    elif 18 <= h <= 23:
        return 'Noche'
    else:
        return 'Madrugada'

df['FRANJA_HORARIA'] = df['HORA'].apply(clasificar_franja_horaria)


In [9]:
# Paso 1: Calcular comisión
df['COMISION_CALCULADA'] = (df['TOTAL DE LA ORDEN'] * df['% COMISION DE LA PLATAFORMMA']) / 100

# Paso 2: Calcular ganancia estimada
df['GANANCIA_CALCULADA'] = (
    df['TOTAL DE LA ORDEN'] 
    - df['PRECIO PROVEEDOR X CANTIDAD']
    - df['PRECIO FLETE']
    - df['COMISION_CALCULADA']
)

# Paso 3: Rellenar valores nulos en GANANCIA
df['GANANCIA'] = df['GANANCIA'].fillna(df['GANANCIA_CALCULADA'])

# Limpieza: eliminar columnas auxiliares si no las necesitas
df.drop(columns=['GANANCIA_CALCULADA', 'COMISION_CALCULADA'], inplace=True)

# Margen de ganancia (%)
df['MARGEN_PORCENTAJE'] = (df['GANANCIA'] / df['TOTAL DE LA ORDEN']) * 100

# Ganancia por unidad
df['GANANCIA_POR_UNIDAD'] = df['GANANCIA'] / df['CANTIDAD']

# ROI (%)
df['ROI'] = (df['GANANCIA'] / df['PRECIO PROVEEDOR X CANTIDAD']) * 100


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 479 entries, 0 to 479
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   ID                            479 non-null    int64         
 1   HORA                          479 non-null    object        
 2   FECHA                         479 non-null    datetime64[ns]
 3   NOMBRE CLIENTE                479 non-null    object        
 4   ESTATUS                       479 non-null    object        
 5   TIPO DE ENVIO                 479 non-null    object        
 6   DEPARTAMENTO DESTINO          479 non-null    object        
 7   CIUDAD DESTINO                479 non-null    object        
 8   TRANSPORTADORA                479 non-null    object        
 9   TOTAL DE LA ORDEN             479 non-null    int64         
 10  GANANCIA                      479 non-null    float64       
 11  PRECIO FLETE                  479 non

In [11]:
df.describe(include='all') 

,ID,HORA,FECHA,NOMBRE CLIENTE,ESTATUS,TIPO DE ENVIO,DEPARTAMENTO DESTINO,CIUDAD DESTINO,TRANSPORTADORA,TOTAL DE LA ORDEN,...,PRECIO PROVEEDOR X CANTIDAD,PRODUCTO,CANTIDAD,DIA_SEMANA,MES,AÑO,FRANJA_HORARIA,MARGEN_PORCENTAJE,GANANCIA_POR_UNIDAD,ROI
count,4.790000e+02,479,479,479,479,479,479,479,479,479.000000,...,479.000000,479,479.000000,479,479,479.0,479,479.000000,479.000000,479.000000
unique,NaN,385,NaN,371,23,2,25,146,9,NaN,...,NaN,8,NaN,7,3,NaN,4,NaN,NaN,NaN
top,NaN,13:30:00,NaN,Alexander Guerrero,ENTREGADO,CON RECAUDO,CUNDINAMARCA,BOGOTA,INTERRAPIDISIMO,NaN,...,NaN,Arlex Rizos Pro,NaN,Saturday,May,NaN,Tarde,NaN,NaN,NaN
freq,NaN,4,NaN,7,218,472,166,118,237,NaN,...,NaN,441,NaN,103,258,NaN,167,NaN,NaN,NaN
mean,4.108767e+07,NaN,2025-05-11 15:04:53.110647296,NaN,NaN,NaN,NaN,NaN,NaN,76470.167015,...,24386.012526,NaN,1.146138,NaN,NaN,2025.0,NaN,46.346264,32438.146841,151.164633
min,3.863824e+07,NaN,2025-04-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,27450.000000,...,12000.000000,NaN,1.000000,NaN,NaN,2025.0,NaN,0.000000,0.000000,0.000000
25%,4.023896e+07,NaN,2025-04-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,64900.000000,...,20000.000000,NaN,1.000000,NaN,NaN,2025.0,NaN,41.262712,26779.500000,133.897500
50%,4.070484e+07,NaN,2025-05-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,64900.000000,...,20000.000000,NaN,1.000000,NaN,NaN,2025.0,NaN,46.271186,31329.500000,152.997500
75%,4.172814e+07,NaN,2025-05-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,72900.000000,...,20000.000000,NaN,1.000000,NaN,NaN,2025.0,NaN,50.430663,34419.500000,170.292500
max,4.342139e+07,NaN,2025-06-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,415206.000000,...,60000.000000,NaN,3.000000,NaN,NaN,2025.0,NaN,77.388508,107107.243333,535.536217


In [12]:
df.to_csv("../output/historial_pedidos.csv", index=False)